In [1]:
import pandas as pd

In [2]:
from gensim.models import KeyedVectors

c:\programdata\anaconda3\envs\mlenv\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [24]:
import pickle

In [28]:
with open('wmd_model.pkl', 'wb') as f:
    pickle.dump(model, f)

MemoryError: 

In [3]:
%%time
model = KeyedVectors.load_word2vec_format('D:/wiki.en.vec', binary=False)

Wall time: 50min 12s


In [4]:
input_text = pd.read_csv('D:/baklazhan/Compling/rec_graph/graph_id_clean_text.csv')

In [5]:
test_x_y = pd.read_csv('D:/baklazhan/Compling/rec_graph/test_links_class_similarities.csv')

In [6]:
test_x_y.head()

,citing,cited,class,sim_d2v,sim_tfidf,sim_ft_mean,sim_ft_weighted
0,C14-1216,W07-1019,0,0.454362,0.004421,0.853234,0.785374
1,W13-2002,D08-1027,0,0.547518,0.029428,0.870538,0.838423
2,D13-1041,W14-1304,0,0.604749,0.011548,0.867591,0.818291
3,P14-2095,P13-2017,1,0.260585,0.133643,0.870482,0.854460
4,C14-1089,W04-3217,0,0.505673,0.025127,0.895686,0.848893


In [7]:
input_text.head()

,id,node_id,abstract
0,A00-1002,0,using examples transfer based mt system czech ...
1,A00-1005,1,paper describes system provides customer servi...
2,A00-1006,2,paper proposes way improve translation quality...
3,A00-1008,3,paper describes application ape atlas planning...
4,A00-1009,4,paper describe implemented framework developin...


In [8]:
input_text[input_text['id'] == test_x_y['citing'][0]]['abstract'].values[0]

'examine task separating types brands food domain framing problem ranking task convert simple textual features extracted domain specific corpus ranker without need labeled training data method rank brands e g sprite higher types e g lemonade apart also exploit knowledge induced semisupervised graph based clustering two different purposes one hand produce auxiliary categorization food items according food guide pyramid assume food item type belongs category unlikely contain brands hand directly model task brand detection using seeds provided output textual ranking features also harness wikipedia articles additional knowledge source'

In [9]:
distance = model.wmdistance('rain deer is lost in the forest', 'animal can not find his way in the wood')

In [10]:
distance

1.6763979775179965

In [11]:
distance = model.wmdistance(input_text[input_text['id'] == test_x_y['citing'][0]]['abstract'].values[0], 
                            input_text[input_text['id'] == test_x_y['cited'][0]]['abstract'].values[0])

In [12]:
distance

0.6425765099287455

In [13]:
model.wmdistance(input_text[input_text['id'] == test_x_y['citing'][3]]['abstract'].values[0], 
                            input_text[input_text['id'] == test_x_y['cited'][3]]['abstract'].values[0])

0.652218298518323

In [14]:
f_wmd = lambda x: model.wmdistance(input_text[input_text['id'] == x['citing']]['abstract'].values[0], 
                            input_text[input_text['id'] == x['cited']]['abstract'].values[0])

In [15]:
%%time
test_x_y['wmd'] = test_x_y.apply(f_wmd, axis = 1)

Wall time: 7min 38s


In [16]:
test_x_y[test_x_y['class'] == 0]['wmd'].min()

0.21120697425291585

In [17]:
test_x_y[test_x_y['class'] == 1]['wmd'].min()

0.0

In [18]:
test_x_y[test_x_y['class'] == 0]['wmd'].mean()

0.6046992590197461

In [19]:
test_x_y[test_x_y['class'] == 1]['wmd'].mean()

0.5546026097937531

In [20]:
test_x_y[test_x_y['class'] == 0]['wmd'].max()

1.8882284424622313

In [21]:
test_x_y[test_x_y['class'] == 1]['wmd'].max()

1.32364617616696

In [35]:
from gensim.similarities import WmdSimilarity

In [36]:
%%time
wmd_sim = WmdSimilarity(input_text['abstract'], model, num_best=10)

Wall time: 1min 17s


In [46]:
similar = wmd_sim[input_text['abstract'].values[1080]]

In [47]:
similar

[(1080, 1.0),
 (7337, 0.9294027208588971),
 (9035, 0.9264838335285507),
 (2397, 0.9262759730543527),
 (560, 0.9259126325307288),
 (12993, 0.9245206684111129),
 (13381, 0.924468473188548),
 (13041, 0.9240026743588734),
 (12680, 0.9234929087642102),
 (4488, 0.9233397975523558)]

In [45]:
input_text[input_text['id'] == 'C14-1216']

,id,node_id,abstract
1080,C14-1216,1080,examine task separating types brands food doma...


In [48]:
test_x_y[(test_x_y['citing'] == 'C14-1216')|(test_x_y['cited'] == 'C14-1216')]

,citing,cited,class,sim_tfidf,sim_ft_mean,sim_ft_weighted,wmd
0,C14-1216,W07-1019,0,0.004421,0.853234,0.785374,0.642577
30,C14-1216,P12-1057,1,0.020812,0.915617,0.872597,0.629647
3036,C14-1216,P97-1023,1,0.028453,0.871837,0.816986,0.689834
10712,W13-1907,C14-1216,0,0.005649,0.877624,0.846649,0.559608
10993,C14-1216,N12-2012,0,0.049339,0.913325,0.849708,0.502507
16049,C14-1216,W97-0802,1,0.000000,0.862972,0.832919,0.664523
18239,C14-1216,P06-1123,0,0.015826,0.884056,0.838147,0.603928
18672,C14-1216,C96-1079,0,0.003912,0.844038,0.800574,0.521935
18937,C14-1216,P98-2182,1,0.052143,0.925693,0.896744,0.471055
22255,C14-1216,W04-1107,0,0.012593,0.890886,0.837131,0.688427


In [22]:
test_x_y.head()

,citing,cited,class,sim_d2v,sim_tfidf,sim_ft_mean,sim_ft_weighted,wmd
0,C14-1216,W07-1019,0,0.454362,0.004421,0.853234,0.785374,0.642577
1,W13-2002,D08-1027,0,0.547518,0.029428,0.870538,0.838423,0.586771
2,D13-1041,W14-1304,0,0.604749,0.011548,0.867591,0.818291,0.650556
3,P14-2095,P13-2017,1,0.260585,0.133643,0.870482,0.854460,0.652218
4,C14-1089,W04-3217,0,0.505673,0.025127,0.895686,0.848893,0.756907


In [23]:
test_x_y.to_csv('D:/baklazhan/Compling/rec_graph/test_links_class_similarities_2505.csv', index=False)

In [49]:
with open('D:/baklazhan/Compling/rec_graph/pctadw.embedding', 'r') as f:
    pctadw_emb = f.read()

In [54]:
pctadw_emb = pctadw_emb.split('\n')

In [64]:
import decimal
import numpy as np

In [65]:
np.array([float(num) for num in pctadw_emb[0].split()])

array([-8.93860706e-04,  1.60982266e-01, -2.11674064e-01, -2.74693906e-01,
        1.97055027e-01,  1.61276862e-01,  1.20479964e-01,  2.94525474e-01,
       -4.81311381e-02,  1.04785196e-01, -1.52435720e-01,  7.38123083e-04,
       -1.14586137e-01, -2.90383846e-01,  1.60017416e-01,  3.18332613e-01,
        4.74042259e-02, -1.24664173e-01, -5.38795181e-02,  1.15151308e-01,
        2.01538298e-02,  1.08358063e-01,  1.23255700e-01,  1.28390476e-01,
       -2.92334795e-01, -3.55916508e-02, -1.81984767e-01, -1.61383390e-01,
       -1.00058503e-01,  2.84194499e-01,  2.39430685e-02,  5.53812012e-02,
       -2.80262833e-03, -2.03434184e-01, -8.76062512e-02, -1.29554287e-01,
       -2.05357522e-02, -1.67665467e-01, -4.10764255e-02,  2.06701815e-01,
        6.05085492e-02,  1.19083256e-01, -1.05225518e-01, -2.40415886e-01,
       -6.46040812e-02,  2.47369677e-01, -2.74373777e-02,  2.47500286e-01,
        6.29119799e-02, -8.84795040e-02, -2.46053636e-02,  9.01527628e-02,
        1.66662619e-01, -